Export and import venv: https://stackoverflow.com/questions/14684968/how-to-export-virtualenv

<font size = 30>Collecting all packages

In [ ]:

import geopandas as gpd
import matplotlib.pyplot as plt  
import matplotlib.animation 
import numpy as np
import requests 
import os
import pandas as pd
import psycopg2
import contextily as ctx
from shapely.geometry import Point
from mpl_toolkits.axes_grid1 import make_axes_locatable
from itertools import chain
from datetime import date
from typing import List
import pathlib

<font size = 30>Read and Prepare Patentdata for further modifications

In [ ]:

#read files inside folder for patent data

wd = os.getcwd()
wd2 = wd + "/Patentdata"
def find_excel_files_in(directory:pathlib.Path) -> List[pathlib.Path]:
    files:List[pathlib.Path] = list()
    for filepath in directory.rglob('*.xlsx'):
        if filepath.is_file():
            files.append(filepath)

    return files

# List of your directories
directories:List[str] = [format(wd2)]

found_files:List[pathlib.Path] = list()

for directory in directories:
    directory:pathlib.Path = pathlib.Path(directory)
    found_files.extend(find_excel_files_in(directory))

#read excel files
listoffiles=[]
for count,files in enumerate(found_files):
    globals()[f"data{count}"] = pd.read_excel(files)
    listoffiles.append(globals()[f"data{count}"])

if len(listoffiles) > 1:
    data = pd.concat(listoffiles)
else:
    data = globals()[f"data{count}"]

#columnnames: delete "-"
data.columns = data.columns.str.replace("-","")

#columnnames: replace double whitespace with "_"
data.columns = data.columns.str.replace("  ","_")

#columnnames: lowercase and replace whitespace with "_"
data.columns= data.columns.str.lower()
data.columns = data.columns.str.replace(" ","_")

#columnnames: replace "/" with "_or_"
data.columns = data.columns.str.replace("/","_or_")

#columnnames: delete ","
data.columns = data.columns.str.replace(",","")

#columnnames: replace "=" with "equals"
data.columns = data.columns.str.replace("=","equals")

#specific columnvalues: apply lowercase
specific_columns = ["title","assignee_or_applicant","optimized_assignee","inventor","abstract","claims",
                    "dead_or_alive_inpadoc_family_status"]

for i in range(len(specific_columns)):
    data[f"{specific_columns[i]}"]=data[f"{specific_columns[i]}"].str.lower()

#specific columnvalues: to string
specific_columns.append("inpadoc_family_members")
for i in range(len(specific_columns)):
    data[f"{specific_columns[i]}"]=data[f"{specific_columns[i]}"].astype("string")

#delete duplicate publication_numbers
data = data.drop_duplicates(subset="publication_number", keep='last')

#reset index
data.reset_index(inplace= True)
data = data.drop("index", axis = 1)



<font size = 30>Read and prepare translationaldata in order to modify patentdata

In [ ]:

#read files inside folder for translation of publication Number 

wd = os.getcwd()
wd2 = wd + "/TranslationData"
def find_excel_files_in(directory:pathlib.Path) -> List[pathlib.Path]:
    
    files:List[pathlib.Path] = list()
    for filepath in directory.rglob('*.xlsx'):
        if filepath.is_file():
            files.append(filepath)

    return files

# List of your directories
directories:List[str] = [format(wd2)]

found_files:List[pathlib.Path] = list()

for directory in directories:
    directory:pathlib.Path = pathlib.Path(directory)
    found_files.extend(find_excel_files_in(directory))

#read excel files

for files in found_files:
    xls = pd.ExcelFile(files)

file = pd.read_excel(xls, "PUBLNR",skiprows=2)

filter = file["DOCDB"] != " "
file = file[filter]

#Filling all empty cells in the country code(CC) column inside file df

for x in range(len(file)):
    if file.iloc[x][0] != " ":
        shortcode_for_country = file.iloc[x][0]
    if file.iloc[x][0] == " ":
        file.iloc[x][0] = shortcode_for_country
    if file.iloc[x][1] != " ":
        countryname = file.iloc[x][1]
    if file.iloc[x][1] == " ":
        file.iloc[x][1] = countryname




#read ST.30 Sheet in excel file for creating "simplified_status",patent_status and patent_status_description
df1_additional = pd.read_excel(xls, 'ST30_Pub availability',skiprows = 2)
df1_additional = df1_additional[["Unnamed: 0", "Kindcode", "ST.30"]].astype("string").fillna(" ")
df1_additional.reset_index(inplace= True)
df1_additional = df1_additional.drop("index", axis = 1)
df1_additional["Kindcode"] = df1_additional["Kindcode"].str.replace(" ","")
#Filling all empty cells in the country code(CC) column

for x in range(len(df1_additional)):
    if df1_additional.iloc[x][0] != " ":
        shortcode_for_country = df1_additional.iloc[x][0]
    if df1_additional.iloc[x][0] == " ":
        df1_additional.loc[df1_additional.index[x], 'Unnamed: 0'] = f"{shortcode_for_country}"

#Concatenating all different types of documents into one Column

file = file.groupby(['CC','Description','DOCDB'])['Type of document'].apply(', '.join).reset_index()

for col in ["Type of document"]:
    file[col]=file[col].str.split(", ").map(set).str.join(", ")
file["Type of document"] = file["Type of document"].str.replace(', ','| ')

#Translation summary of the type of document
df1_translation = pd.read_excel(xls, 'ST30_Pub availability',skiprows = 0)
df1_translation = df1_translation[["ST.30","Description XML element"]].astype("string").fillna(" ")
filter = df1_translation["ST.30"] != " "
df1_translation = df1_translation[filter]

<font size = 30>modification of patentdata using translationaldata

In [ ]:

#Comparing columns from both data frames(file and data) to fill the column 'Country' in the data df
data["Country"] = ""
liste1=[]
fileCC = list(file.CC)
fileDescription = list(file.Description)
datapubnum = list(data["publication_number"].str[0:2])
for i in range(len(data.publication_number)):
    for x in range(len(fileCC)):    
        if datapubnum[i]== fileCC[x]:
            data.loc[i,'Country'] = file.loc[x,"Description"]
            break
#Inserting a "Type of document" column
#Comparing the last two indexes to the DOCDB column to determine the type of document
#Comparing the last index to the DOCDB column to determine the type of document
data["type_of_document"] = ""
liste2=[]
fileDOCDB = list(file.DOCDB)
data_Country = list(data.Country)
#file_description = list(file.Description)
file_type_of_doc = list(file["Type of document"])
datapubnum1 = list(data["publication_number"].str[-2:])
datapubnum_1 = list(data["publication_number"].str[-1:])

for i in range(len(data.publication_number)):
    for x in range(len(fileDOCDB)):
        if datapubnum1[i]  == fileDOCDB[x] and data_Country[i] == fileDescription[x]:
            data.loc[i,"type_of_document"] = file.loc[x,"Type of document"]
            break
        if datapubnum_1[i]  == fileDOCDB[x] and data_Country[i] == fileDescription[x]:
            data.loc[i,"type_of_document"] = file.loc[x,"Type of document"]
            break   
#Making a new column comparing two df
#The Kindcode from df1_additional df and publication number from the data df
#Numbers compared and inserted(450,470, A44 etc), each number represents a type
#of document
data["ST.30"] = ""
liste3= []
df1_additional_ST_30 = list(df1_additional["ST.30"])
df1_additional_Unnamed = list(df1_additional["Unnamed: 0"])
df1_additional_Kindcode = list(df1_additional.Kindcode)
for i in range(len(data.publication_number)):
    for x in range(len(df1_additional)):
        if datapubnum1[i]  == df1_additional_Kindcode[x]:
            if datapubnum[i] == df1_additional_Unnamed[x]: 
                data.loc[i,'ST.30'] = df1_additional.loc[x,'ST.30']
                break
        if datapubnum_1[i]  == df1_additional_Kindcode[x]:
            if datapubnum[i] == df1_additional_Unnamed[x]:
                data.loc[i,'ST.30'] = df1_additional.loc[x,'ST.30']
                break
#create simplified_status column; look at df1_translation for changes
data["simplified_status"]= data.apply(lambda x: "granted" if x["ST.30"]== "450" or x["ST.30"]=="470"
                                                         else "not granted",axis=1)
                            
data["patent_status_description"] = ""
data_ST_30 = list(data["ST.30"])
df1_translation_ST_30 = list(df1_translation["ST.30"])
for i in range(len(data.publication_number)):
    for x in range(len(df1_translation)):
  
        if data_ST_30[i]  == df1_translation_ST_30[x]:
            data.loc[i,"patent_status_description"] = df1_translation.loc[x,'Description XML element']
            break

In [ ]:

#elimniate brackets (perhaps this is already done before)
data.rename(columns={'claims_(english)': 'claims_english'}, inplace=True)
#make claims lowercase
data["claims_english"] = data["claims_english"].str.lower()
#not 100% sure if this one is necessary
# data.claims_english = data.claims_english.fillna('')
# make a split so we have each claim on its own
data["claims_english"] = data.apply(lambda x: str(x["claims_english"]).split(" | "),axis = 1)
# function to delete entries in claims, where first element in list was just a string and no actual claim
data["claims_english"] = data.apply(lambda x : x["claims_english"] if "1" in x["claims_english"][0] else x["claims_english"][1:], axis=1)


<font size = 30>Modification for claims(english) inside patentdata

In [ ]:

#get a new df with the necessary information
data_claims = data[["publication_number", "claims_english"]]
#Create column CC so we can filter for US (this was for a view only and can be replaced with Musas Code later)
data_claims["CC"] = data_claims["publication_number"].str[:2]
#Explode the dataframe, so we have a row for each single claim
data_claims = data_claims.explode("claims_english").reset_index(drop=True)
#create new columns with the count of the words in combs
combs=["use","method","mixture","process"]

for i in combs:

    data_claims[f"count_{i}"] = data_claims.claims_english.str.count(f"{i}")

#create columns which indicate which type of claim it is
data_claims["use"] = data_claims.apply(lambda x: int(1) if x["count_use"] > 0 and x["CC"] != "US" else 0,axis = 1)
data_claims["method"] = data_claims.apply(lambda x: int(1) if (x["count_method"] > 0 or x["count_process"] > 0) and x["use"] == 0 
                                                           else 0,axis = 1)
data_claims["mixture"] = data_claims.apply(lambda x: int(1) if x["use"] == 0 and x["method"] == 0 and pd.notnull(x["claims_english"]) 
                                                            else 0,axis = 1)
#groupby the publication number, so we have numbers od different claim type for each patent
data_claims_final = data_claims.groupby("publication_number").sum()
#drop the counters and country code
data_claims_final = data_claims_final.drop(columns = ["count_use", "count_method", "count_process", "count_mixture"])
#join the numbers of claim type with the original data (needs to be adjusted according to dataframe names)
data = data.join(data_claims_final, on = "publication_number")
#add the number of claims in each patent (this can or could be done before perhaps)
data["len_claims"] = data.claims_english.str.len()

<font size = 30>Check if some patents are missing inside inpadoc_family_members

In [ ]:
data["inpadoc_family_members"] = data["inpadoc_family_members"].fillna("")
data["inpadoc_family_members"] = data.apply(lambda x: x["inpadoc_family_members"] 
                                                        if len(x["inpadoc_family_members"]) > 4 
                                                        else "",axis=1 )
#check if publication number has been added to inpadoc_family_members and if not add it
data["inpadoc_family_members"] = data.apply(lambda x: str(x["publication_number"])+ " | " +str(x["inpadoc_family_members"]) 
                                            if str(x["publication_number"]) not in str(x["inpadoc_family_members"]) else str(x["inpadoc_family_members"])
                                            ,axis = 1)
#prepare inpadoc_family_members_column
data["inpadoc_family_members"] = data.apply(lambda x: str(x["inpadoc_family_members"]).split(" | "),axis = 1)

#remove entries which are too short in inpadoc_family_members
for x in range(len(data)):
    cell = data.loc[x,"inpadoc_family_members"]
    for k in cell:
        if len(k) < 5:
            cell.remove(k)
    if cell != data.loc[x,"inpadoc_family_members"]:
        data.loc[x,"inpadoc_family_members"] = cell

#x is a list of publicationnumbers
#df is the dataframe
def find_members(x,df):
    #create empty lists for loop
    global inpadoc_undo_nested
    global inpadoc_family_members_find_list
    global add_members
    add_members = ""
    inpadoc_family_members_find_list = []
    inpadoc_undo_nested = []
    #checklist with all publicationnumbers
    checklist = str(df["publication_number"])
    for k in x:
        #filter possible false entries inside column
        if len(k) > 4:
            inpadoc_family_members_find_list.append(k)
        else:
            continue
        if k in checklist:
            #unlist publicationnumbers inside patentfamily
            family_members = list(chain.from_iterable(list(df.loc[df["publication_number"]==str(k),"inpadoc_family_members"])))
            for members in family_members:
                #filter possible false entries inside column
                if len(members) > 4:
                    inpadoc_family_members_find_list.append(members)
                    list(chain.from_iterable(inpadoc_family_members_find_list))
                else:
                    continue
        else:
            continue
    return inpadoc_family_members_find_list
#This function includes the "find_members" function. It might be the case that the "inpadoc_family_members"-column is incomplete
#for some patents. The following function searches through every patentfamily for each publication number and creates a common list for
#all patents inside a patentfamily. This Code could also be used if a new row is being added with an updated version of the patent family. 
#This will lead to an update for all previous patents inside the data.
def iteration_of_find_members(x,df):
    global inpadoc_undo_nested
    global inpadoc_family_members_find_list
    inpadoc_family_members_find_list = []
    inpadoc_undo_nested = []  
    global run0
    run0 = find_members(x,df)
    run1 = find_members(run0,df)
    test1 = run0
    test2 = run1
    k= 0
    #This function searches for patents inside a family as long as the list doesn't get longer anymore
    while set(test1) != set(test2):
        globals()[f"run1"] = find_members(run0,df)
        test1 = run1
        globals()[f"run0"] = find_members(run1,df)
        test2 = run0
        k=+1
    return list(set(test2))

#creating len of inpadoc_family_members
data["len_inpadoc"] = data.apply(lambda x: len(x["inpadoc_family_members"]),axis = 1)
#creating copy of data df for check up
test = data
test["test"] = test.apply(lambda x: iteration_of_find_members(x["inpadoc_family_members"],test),axis = 1)
test["len_inpadoc"] = test.apply(lambda x: len(x["inpadoc_family_members"]),axis = 1)
test["len_test"] = test.apply(lambda x: len(x["test"]),axis = 1)

<font size = 30>Export Dataframe to /Prepared_Data folder in .csv and .xlsx format

In [ ]:

data.to_csv(f"{wd+'/Prepared_Data'}/generated_data_{str(date.today()).replace('-','_')}.csv", index=False)

In [ ]:

data.to_excel(f"{wd+'/Prepared_Data'}/generated_data_{str(date.today()).replace('-','_')}.xlsx", index=False)